# Generate EAP for *bifurcation* model

Note: this notebook can be run only once. To generate a new simulation you have to restart the kernel (Kernel->Restart)

In [ ]:
import neuron
from math import sin, cos
import numpy as np
import LFPy
import MEAutility as mu
import matplotlib.pyplot as plt
import neuroplotlib as npl
from pathlib import Path
import sys
from pprint import pprint
import os

In [ ]:
from axon_velocity.models import create_morph_with_axon_bifurcations, insert_biophysics, \
    insert_simple_biophysics, save_cell, create_mea_probe, get_default_biophysics_params

In [ ]:
#%matplotlib widget
%matplotlib notebook

In [ ]:
save_fig = True
save_results = False

In [ ]:
try:
    import neuron
except:
    print('NEURON is not installed.')

mechanism_folder = Path('..') / 'mechanisms'

if not neuron.load_mechanisms(str(mechanism_folder)):
    print('Compile mod files in the mechanisms/ folder: from the mechanisms/ folder, run nrnivmodl')

In [ ]:
d_axon = 0.5
z_offset = 10

In [ ]:
# simple biophysiscs: dendrite - pas / soma/axon HH
# "complex" biophysics: dendrite - pas / soma - na + kv1 / axon - nax + kv1 
simple_biophysics = False

In [ ]:
params_dict = get_default_biophysics_params()
pprint(params_dict)

At this stage, one can also change the axial conductance (e.g. `sec.ra`), 
which likely affects the conduction velocity.

In [ ]:
z_offset = 10

In [ ]:
cell, secs = create_morph_with_axon_bifurcations(nseg=100, v_init=params_dict['v_init'], 
                                                 celsius=params_dict['celsius'],
                                                 Ra=params_dict['ra'], cm=params_dict['cm'], d_axon=d_axon)

In [ ]:
ax = npl.plot_detailed_neuron(cell, plane='xy', color_axon='g', color_childaxon1='g', color_childaxon2='g')

In [ ]:
if save_fig:
    fig = ax.get_figure()
    fig_folder = Path('..') / 'figures'
    if not fig_folder.is_dir():
        os.makedirs(fig_folder)
    fig.savefig(fig_folder / 'bifurcation.pdf')

### Insert cell biophysics

Here we make the cell active by inserting biophysical mechanisms.

In [ ]:
if simple_biophysics:
    insert_simple_biophysics(cell)
else:
    insert_biophysics(cell, params_dict)

### Stimulating the cell

We can now add some stimulation. The stimulation can be a current clamp `iclamp` or synaptic inputs `syn`. The `stim_point` is where the cell will be stimulated (the closest cell segment to the `stim_point` is used).

In [ ]:
stim = 'syn' # or syn
stim_point = [0, 0, 300]
stim_idx = cell.get_closest_idx(x=stim_point[0], y=stim_point[1], z=stim_point[2])

syn_input_times = [5]
syn_params = {'idx' : stim_idx,
              'e' : 0,                                # reversal potential
              'syntype' : 'ExpSyn',                   # synapse type
              'tau' : 2,                              # syn. time constant ms
              'weight' : 0.02,                        # syn. weight
              'record_current' : True                 # syn. current record
    }
clamp_params = {'idx' : stim_idx,
                'pptype' : 'IClamp',                   # IClamp point process
                'dur' : 30,                            # dur in ms
                'amp' : 0.2,                           # amp in nA
                'delay' : 5                            # delay in ms
    }

#%%

if stim == 'syn':
    synapse = LFPy.Synapse(cell, **syn_params)
    synapse.set_spike_times(np.array(syn_input_times))
else:
    clamp = LFPy.StimIntElectrode(cell=cell, **clamp_params)

### Define extracellular electrodes

Let's now define the extracellular electrodes using the [MEAutility](https://meautility.readthedocs.io/en/latest/) package.

In [ ]:
mea_dim = 100  # n rows x n cols
mea_pitch = 17.5  # rows and cols pitch
elec_size = 5

hdmea = create_mea_probe(pitch=mea_pitch, dim=mea_dim, elec_size=elec_size, z_offset=z_offset)

electrode = LFPy.RecExtElectrode(cell, probe=hdmea, n=10)

# Instantiate LFPy electrode object
electrode = LFPy.RecExtElectrode(cell, probe=hdmea)

In [ ]:
mu.plot_probe(hdmea)

### Run the simulation

By passing the `electrode` argument `LFPy` also computes extracellular potentials. The `rec_vmem` argument allows to measure the membrane potenrtial at all segments.

In [ ]:
cell.simulate(probes=[electrode], rec_vmem=True)

In [ ]:
eap = electrode.data * 1000  # mV --> uV

In [ ]:
print(eap.shape)

### Plot membrane potentials

In [ ]:
soma_idx = cell.get_closest_idx(0, 0, 0)
dend_idx = cell.get_closest_idx(0, 100, 0)
axon_idx = cell.get_closest_idx(0, -200, 0)

In [ ]:
plt.figure()
plt.plot(cell.vmem[soma_idx], label='soma')
plt.plot(cell.vmem[dend_idx], label='dend')
plt.plot(cell.vmem[axon_idx], label='axon')
plt.legend()

In [ ]:
# cutout single template
fs = 1 / cell.dt
ms_before = 2
ms_after = 10

min_chan, min_idx = np.unravel_index(np.argmin(eap), eap.shape)

In [ ]:
eap_cut = eap[:, min_idx - int(ms_before * fs): min_idx + int(ms_after * fs)]

In [ ]:
ax = mu.plot_mea_recording(eap_cut, hdmea, colors='gray')
npl.plot_detailed_neuron(cell, ax=ax, plane='xy', color='k')

### Plot peak latency map

In [ ]:
min_map = np.argmin(eap_cut, 1).astype('float')
min_idx_cut = np.unravel_index(np.argmin(eap_cut), eap_cut.shape)[1]
min_map[min_map < min_idx_cut] = np.nan
min_map -= min_idx_cut

In [ ]:
plt.matshow(min_map.reshape(mea_dim[1], mea_dim[0]).T, origin='lower')
plt.colorbar()

### Save templates and locations

In [ ]:
template = eap_cut
locations = hdmea.positions[:, :2]  # save only x-y positions

In [ ]:
# save templates and locations
if save_results:
    d_axon_str = f"{d_axon}".replace('.', '-')
    data_folder = Path('..') / 'simulated_data' / 'toy'
    save_path = data_folder / 'bifurcation' / f'diam{d_axon_str}'

    if not save_path.is_dir():
        os.makedirs(save_path)

    np.save(save_path / 'template.npy', template)
    np.save(save_path / 'locations.npy', locations)
    save_cell(cell, cell_name='bifurcation', save_folder=save_path)